In [0]:
filepath = 'dbfs:/FileStore/Global_Retail/Bronze_Layer/Transaction/transaction_snappy.parquet'
df = spark.read \
     .format('parquet') \
     .load(filepath)
df.show(10)

+--------------+-----------+----------+--------+------------+----------------+--------------+--------------+
|transaction_id|customer_id|product_id|quantity|total_amount|transaction_date|payment_method|    store_type|
+--------------+-----------+----------+--------+------------+----------------+--------------+--------------+
|     TRX000001|        802|       425|       1|       363.4|      2020-07-27|    Debit Card|Physical Store|
|     TRX000002|        858|       280|       6|      758.18|      2022-08-10|   Credit Card|Physical Store|
|     TRX000003|        658|       694|       9|      748.66|      2020-05-22| Bank Transfer|        Online|
|     TRX000004|        516|       930|       4|      933.78|            NULL| Bank Transfer|Physical Store|
|     TRX000005|        368|       104|      10|      137.28|      2022-06-24|        PayPal|Physical Store|
|     TRX000006|        606|       409|       6|      556.88|      2020-10-14|   Credit Card|        Online|
|     TRX000007|   

In [0]:
df.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- store_type: string (nullable = true)



In [0]:
from pyspark.sql.functions import to_timestamp, col
new_df = df.withColumn("transaction_date",to_timestamp(col('transaction_date')))

In [0]:
new_df.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- transaction_date: timestamp (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- store_type: string (nullable = true)



In [0]:
from pyspark.sql.functions import current_timestamp
final_df = new_df.withColumn("ingestion_timestamp", current_timestamp())
final_df.show(10)

+--------------+-----------+----------+--------+------------+-------------------+--------------+--------------+--------------------+
|transaction_id|customer_id|product_id|quantity|total_amount|   transaction_date|payment_method|    store_type| ingestion_timestamp|
+--------------+-----------+----------+--------+------------+-------------------+--------------+--------------+--------------------+
|     TRX000001|        802|       425|       1|       363.4|2020-07-27 00:00:00|    Debit Card|Physical Store|2024-10-08 23:52:...|
|     TRX000002|        858|       280|       6|      758.18|2022-08-10 00:00:00|   Credit Card|Physical Store|2024-10-08 23:52:...|
|     TRX000003|        658|       694|       9|      748.66|2020-05-22 00:00:00| Bank Transfer|        Online|2024-10-08 23:52:...|
|     TRX000004|        516|       930|       4|      933.78|               NULL| Bank Transfer|Physical Store|2024-10-08 23:52:...|
|     TRX000005|        368|       104|      10|      137.28|2022-06-

In [0]:
#saving as a Table now in Delta Table

spark.sql("use globalretail_bronze")

final_df.write \
.format('delta') \
.mode('append') \
.saveAsTable('bronze_transactions')

In [0]:
import datetime
archive = 'dbfs:/FileStore/Global_Retail/Bronze_Layer/Transaction/Archive/'
path = archive +'_'+ datetime.datetime.now().strftime("%Y%m%d")
dbutils.fs.mv(filepath, path)
print(path)

---------------------------------------------------------------------------
UnsupportedOperationException             Traceback (most recent call last)
File <command-1387934050476918>, line 4
      2 archive = 'dbfs:/FileStore/Global_Retail/Bronze_Layer/Transaction/Archive/'
      3 path = archive +'_'+ datetime.datetime.now().strftime("%Y%m%d")
----> 4 dbutils.fs.mv(filepath, path)
      5 print(path)

File /databricks/python_shell/dbruntime/dbutils.py:364, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    362 def f_with_exception_handling(*args, **kwargs):
    363     try:
--> 364         return f(*args, **kwargs)
    365     except Py4JJavaError as e:
    367         class ExecutionError(Exception):

File /databricks/python_shell/dbruntime/dbutils.py:419, in DBUtils.FSHandler.mv(self, source, dest, recurse)
    417     return self.print_return(self.fsutils_parallel.mvParallel(source, dest))
    418 else:
--> 419     return self.p